# GCC's ssc-cdi : tomography pipeline for Cateretê v0.1
 
- #### Questions? E-mail gcc@lnls.br
- #### Documentation about the ssc-cdi package: https://gcc.lnls.br/wiki/
- #### To use a test sample:
    - dic["sinogram_path"] = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/glass24/2023-05-12-15h19m_glass24.hdf5"

## Imports

In [ ]:
%matplotlib widget
import numpy as np, matplotlib.pyplot as plt, json
import sscInstall
import sscCdi

## Load dictionary

In [ ]:
dic = {}

In [ ]:
# dic = json.load(open('/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/inputs/CAT_tomo_inputs.json'))
dic['processing_steps']['sort'] = True
dic['processing_steps']['crop'] = True
dic['processing_steps']['equalize2D'] = True
dic['processing_steps']['unwrap'] = True
dic['processing_steps']['alignment'] = True
dic['processing_steps']['tomography'] = True
dic['processing_steps']['equalize3D'] = True
dic

# Run at Cluster

In [ ]:
# server = sscInstall.connect_server()    

In [ ]:
# json_filepath_path = sscCdi.cat_ptycho_processing.save_input_dictionary(dic)
# sscCdi.run_at_cluster(server,json_filepath_path,queue='cat',gpus=dic["GPUs"],cpus=dic["CPUs"],jobName='job',script_path = "/ibira/lnls/labs/tepui/home/yuri.tonin/ssc-cdi/bin/caterete_tomo.py",slurm_path = '/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/inputs/')

# Run Locally

## Load data

In [ ]:
dic["recon_method"]  = "ptycho" # ptycho or pwcdi
dic["contrast_type"] = "phase" # phase or magnitude
dic["sinogram_path"] = "/ibira/lnls/labs/tepui/home/yuri.tonin/00000000/proc/recons/glass_sphere/2023-05-16-15h38m_glass_sphere.hdf5"

In [ ]:
dic = define_paths(dic)

In [ ]:
obj, angles = sscCdi.read_data(dic)
print(f"Object shape = {obj.shape} \t Number of angles: {angles.shape}")

In [ ]:
sscCdi.deploy_visualizer(obj,title='Original sinogram',cmap='gray')

In [ ]:
crop = obj[:,1500:-1400,1400:-1500]
sscCdi.deploy_visualizer(crop,title='Original sinogram',cmap='gray')

## Sort data

In [ ]:
sscCdi.tomo_sort(dic,obj, angles)

In [ ]:
sscCdi.deploy_visualizer(np.load(dic["ordered_object_filepath"]),title="Ordered sinogram",cmap='gray')

In [ ]:
dic["top_crop"]    = 1500
dic["bottom_crop"] = 1400
dic["left_crop"]   = 1400
dic["right_crop"]  = 1500

In [ ]:
sscCdi.tomo_crop(dic)

In [ ]:
sscCdi.deploy_visualizer(np.load(dic["cropped_sinogram_filepath"]),title="Cropped sinogram",cmap='gray')

## Unwrap

In [ ]:
dic["bad_frames_before_unwrap"] = [28,30,45,49,65,66,91,113]

In [ ]:
sscCdi.tomo_unwrap(dic)

In [ ]:
sscCdi.deploy_visualizer(np.load(dic["unwrapped_sinogram_filepath"]),title="Unwrapped sinogram",cmap='gray')

## 2D Equalization

In [ ]:
dic["sinogram_to_equalize"] = 'unwrapped' # cropped or unwrapped
dic["bad_frames_before_equalization"] = []
dic["equalize_gradient"] = True
dic["equalize_global_offset"] = False
dic["equalize_invert"] = True
dic["equalize_local_offset"] = [0,50,0,50]
dic["equalize_outliers"] = 0

In [ ]:
sscCdi.tomo_equalize(dic)

In [ ]:
sscCdi.deploy_visualizer(np.load(dic["equalized_sinogram_filepath"]),title="Equalized sinogram",cmap='gray')

In [ ]:
sscCdi.deploy_visualizer(np.load(dic["equalized_sinogram_filepath"]),title="Equalized sinogram",cmap='gray',axis=1,aspect_ratio='auto')#,limits=(0,0.5))

## Alignment

In [ ]:
dic["wiggle_sinogram_selection"] = dic["equalized_sinogram_filepath"]
dic["step_percentage"] = 0 # need to project irregular angle steps to a regular grid?

In [ ]:
sscCdi.preview_angle_projection(dic)

In [ ]:
dic["CPUs"] = 64
dic["project_angles_to_regular_grid"] = True
dic["bad_frames_before_wiggle"] = [] 
dic["wiggle_reference_frame"] = 0

In [ ]:
dic = sscCdi.tomo_alignment(dic)

In [ ]:
sscCdi.deploy_visualizer(np.load(dic["wiggle_sinogram_filepath"]),title="Aligned sinogram",cmap='gray',axis=1)

## Tomography

In [ ]:
dic["GPUs"] = [0,1]
dic["tomo_algorithm"] = "EEM"
dic["tomo_iterations"] = 10
dic["tomo_regularization_param"] = 0 # if != 0, do regularization. Recommended = 0.001

In [ ]:
recon3D = sscCdi.tomo_recon(dic)

In [ ]:
sscCdi.deploy_visualizer(np.load(dic["reconstruction_filepath"]),title="Recon",cmap='gray',axis=0)#,limits=(-1,1))

## 3D Equalization

In [ ]:
dic["tomo_remove_outliers"] = 0
dic["tomo_threshold"] = 15.0
dic["tomo_local_offset"] = []
dic["tomo_mask"] = []

In [ ]:
sscCdi.tomo_equalize3D(dic)

In [ ]:
sscCdi.deploy_visualizer(np.load(dic["eq_reconstruction_filepath"]),title="Equalized Recon",cmap='gray',axis=0)

In [ ]:
sscCdi.equalized_tomogram = np.load(dic["eq_reconstruction_filepath"])
sscCdi.plot_histograms(recon3D, equalized_tomogram,bins=300)